In [2]:
import pandas as pd
import numpy as np

import requests
from urllib import parse
import json

import time
import datetime

import re

from tqdm import tqdm
from fake_useragent import UserAgent

import warnings

warnings.filterwarnings('ignore')

### Riot API 요청 시간 제한
- 20 requests every 1 seconds(s)
- 100 requests every 2 minutes(s)

#### URL HEADERS

In [ ]:
ua = UserAgent(verify_ssl=False)

In [ ]:
api_key_1 = 'RGAPI-04ce57ff-79b1-4164-96e0-15f3e93135ed' # API 제한기간 1일 https://developer.riotgames.com/ 방문해서 갱신 필ㅇ요
api_key_2 = 'RGAPI-be1645be-f2c3-4009-a982-be109a5dce55'
api_key_3 = 'RGAPI-028f261d-e826-4867-a4df-3a9caf5ce6ab'
api_key_4 = 'RGAPI-29e32243-84d9-4414-bce6-23c2e0f16cde'
api_key_5 = 'RGAPI-8ad794ac-1f89-4ebf-9694-8567073d7246'
# api_key_6 = 'RGAPI-24d70943-4f76-4ba7-ac88-1e4e0ba24b77'
# api_key_7 = 'RGAPI-1acf44d2-1b02-4d36-a3b0-4c7b09f7fc44'
# api_key_8 = 'RGAPI-7615568b-1167-4d44-be04-fe82b34700f6'

api_keys = [api_key_1, api_key_2, api_key_3, api_key_4, api_key_5]

headers = {
    "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com/",
}

#### 소환사 정보 가져오기
 - 소환사 정보에는 3가지 아이디가 존재 소환사 ID, 계정 ID, puuId 
 - 소환사, 계정 -> 지역별 고유 / puuId -> 전세계 고유

In [ ]:
# 소환사 정보 가져오기

def tft_summoner_info_summonerName(summonerName, api_idx): #닉네임으로 탐색
    encodingName = parse.quote(summonerName)
    url = f'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/{encodingName}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

def tft_summoner_info_summonerId(summonerId, api_idx): # 소환사 ID로 탐색
    url = f'https://kr.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers = headers)

def tft_summoner_info_puuid(puuid, api_idx): # puuid로 탐색
    url =  f'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-puuid/{puuid}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers = headers)

#### TFT - 게임 정보 가져오기

In [ ]:
# 타겟 소환사의 게임 id 가져오기. 기본 50 게임 조회
def tft_match_ids(puuid, api_idx, count=50):
    url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

def tft_get_Game_info(matchId, api_idx):
    url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{matchId}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

#### TFT - 티어별 소환사 정보 가져오기

In [ ]:
def tft_tier_summoners(tier, division, api_idx, page=1):
    url = f'https://kr.api.riotgames.com/tft/league/v1/entries/{tier}/{division}?page={page}'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

In [ ]:
# 챌린저, 그마, 마스터 티어는 따로 가져와야함
# 3개의 상위 티어는 페이지 구분없이 해당 티어의 모든 유저의 정보를 넘겨줌

def tft_challenger_summoners(api_idx):
    url = f'https://kr.api.riotgames.com/tft/league/v1/challenger'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

def tft_grandmaster_summoners(api_idx):
    url = f'https://kr.api.riotgames.com/tft/league/v1/grandmaster'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

def tft_master_summoners(api_idx):
    url = f'https://kr.api.riotgames.com/tft/league/v1/master'
    headers["User-Agent"] =  ua.random
    headers["X-Riot-Token"] = api_keys[api_idx]
    return requests.get(url, headers=headers)

## TFT 소환사 정보 가져오기

In [ ]:
summoners_df = pd.DataFrame()

In [ ]:
#나머지 티어 정보 가져오기

def getSummoner_df(): 
    
    results = [pd.DataFrame() for _ in range(5)]

    tiers = ['C', 'GM', 'M', 'DIAMOND','PLATINUM','GOLD', 'SILVER', 'BRONZE', 'IRON']
    divisions = ['I', 'II', 'III', 'IV']

    for tier in tqdm(tiers):
        
        summoners = []
        if tier in ['C', 'GM', 'M']:
            if tier == 'C': #챌린저
                for i in range(5):
                    summoners.append(tft_challenger_summoners(i))

            elif tier == 'GM': #그랜드마스터
                for i in range(5):
                    summoners.append(tft_grandmaster_summoners(i))


            elif tier == "M": #마스터
                for i in range(5):
                    summoners.append(tft_master_summoners(i))


            
            for idx, summoner in enumerate(summoners):
                if idx == 0:
                    texts = json.loads(summoner.text)
                    # 대표 DF
                    top_tier_df = pd.DataFrame(texts['entries'])
                    top_tier_df.sort_values('leaguePoints', ascending=False, inplace=True)
                    top_tier_df['tier'] = texts['tier']
                    top_tier_df['leagueId'] = texts['leagueId']
                    top_tier_df['queueType'] = texts['queue']

                    results[idx] = pd.concat([results[idx], top_tier_df])

                else:
                    texts = json.loads(summoner.text)
                    
                    tier_df = pd.DataFrame(texts['entries'])
                    tier_df.sort_values('leaguePoints', ascending=False, inplace=True)
                    
                    tier_df = tier_df[['summonerId', 'summonerName']]
                    
                    results[idx] = pd.concat([results[idx], tier_df])
            
        else:
            div_df = [pd.DataFrame() for _ in range(5)]
            
            for division in divisions:
                for page in range(1,20):
                    tmp = []
                    for i in range(5):
                        tmp.append(tft_tier_summoners(tier, division, i, page))

                    #소환사 정보가 없는 페이지 확인
                    if tmp[0].text == '[]':
                        break
                    else:
                        for i in range(5):
                            div_df[i] = pd.concat([div_df[i], pd.DataFrame(tmp[i].json())])

                    #요청 제한 시간 범위 때문에 딜레이 설정
                    time.sleep(1)
                    
            for i in range(5):
                if i == 0:
                    results[i] = pd.concat([results[i], div_df[i]])
                else:
                    tmp = div_df[i][['summonerId', 'summonerName']]
                    results[i] = pd.concat([results[i], tmp])
                
    return results

In [ ]:
summoners_list = getSummoner_df()

In [ ]:
for idx, df in enumerate(summoners_list):
    df = df.drop_duplicates(['summonerName'])
    if idx == 0:
        summoner_to_csv = df
    else:
        df = df.rename(columns={'summonerId' : 'summonerId_'+str(idx)})
        
        summoner_to_csv = summoner_to_csv.merge(right=df, how='inner', on = 'summonerName')

In [ ]:
summoner_to_csv.to_csv('./data/concat_puuid_summoner_df.csv')

In [8]:
summoner_to_csv = pd.read_csv('./data/concat_puuid_summoner_df.csv', index_col = 0)
summoner_to_csv.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,leagueId,queueType,summonerId_1,summonerId_2,summonerId_3,summonerId_4
0,2MYs5V2nx0dcmsvyygTGzT-XPVblPF3Zs37vkQi5SQ3QgR...,리롤왕 베인,1917,I,314,167,True,False,False,True,CHALLENGER,bf18e0ba-d906-3a5f-a6f1-62b94453988b,RANKED_TFT,9IUmNZlFv5GswhYypXmGzsjQGgqcG_c-HAJlAqlJhakUoO...,MwL3UvMYo299-UKwstsm_T9fTxm-dO6I-flMbLzeAi2oDK...,hKE-HMCLVnYTXHyTBTUiwdBKtqvrVCiDyfzAlbJV6zS41x...,uUnQtXJyLaeskWQYRAu7nFfH1gkNGjyc_d-ZxJwCCcQil-...
1,S2xKELyJb3YTnwFuFLLHKqY4uaQxNHjEauslOJ-t8Q1lKls,이렇게 좋은 날,1795,I,522,324,True,False,False,True,CHALLENGER,bf18e0ba-d906-3a5f-a6f1-62b94453988b,RANKED_TFT,UlCj_PB1itzGL4Dq6r3VsATmMP5h6JsMYGYpZhkIavYmrR0,2VB7Kbgne2iE_KbSS9hHoAcrwhFChwwgB34OunXfUCqsrRE,E0WthMJVVHh6c6xRGXR3PBm4_81teDWkLSDYdoZ8BUHTMt8,A7tsqYvGyNYPFWV0PDbHbG8lh2XPJA-fdTla7giAFlcvFA4
2,WELtjya8aKAzscRe1o1yQyDtb_wk7C9QZ3iuJ869qH9r9RWj,배불뚝이 팬더,1793,I,344,193,True,False,False,True,CHALLENGER,bf18e0ba-d906-3a5f-a6f1-62b94453988b,RANKED_TFT,83i1sqZoN8YdqfUuyo3wECJQJOzL2EXZ9rPTLwQpSy9rLAtY,uhG03iOhhFKKnDgzQ8YtE6Sp_bN4sF_PSdKBEew7QBy2bM1U,UV7uc0Rh2rtGwJKcIbgXw1AO9r0yfzPCrbhS53Ptlkh5Pqsv,P9I_hAt3CaG8LwDbZdeUrFjFLYvcjspPlsNdtLOEBDODJpvY
3,2oH8DimGYvbzrVHZHbPPo_pLAof6nVk6UKYLlaLy95TsvS...,MOSO,1769,I,270,129,True,False,False,True,CHALLENGER,bf18e0ba-d906-3a5f-a6f1-62b94453988b,RANKED_TFT,TqIJfaR3NrWMGrgn_WPGRmdkZsZUwODMpQnt4JC70YP_Xm...,DRjIDFO9VLg3Llmu2CI_OqU3w_VspV_-lRLkr3_VaE12EZ...,zZ8FghAGmOMQmreb9Oox_TlqnBHzMDgWVhnF58NLkKxHXC...,JntD1bpVlVkKrGtjVKA5GEy--oqigFk254ErNToDpea4bi...
4,NQ8a1sagUeWLsPARNGYHESzr3xaJp80MFEIB0rJ-7U6bDUTB,TTVBebe872,1560,I,281,147,True,False,False,False,CHALLENGER,bf18e0ba-d906-3a5f-a6f1-62b94453988b,RANKED_TFT,kHGtEBH4qLt_xshK6pQq10oXdqOKCa6ehj6mruDt1tSiH3uj,dZz8aoBupvuz_EsxvqrOHPrHNk_CN6Xg5sosAdBm3C2UnyB4,-ZjLFY8RozpfVENoo-KyCpGP9fzF5mb7uOb3exq6cmMbobc-,S_9jRFIq0fUR_HF78g7YiBk-0m4sHdNoZ9EMTKQPTf99zPGj


## 소환사 puuid 가져오기
- 마스터 이하 티어는 계정수가 너무 많아서 1,000개 랜덤 샘플링해서 가져옵니다.

In [ ]:
summoners_df = pd.read_csv('./data/concat_puuid_summoner_df.csv', index_col=0)

In [ ]:
def getPuuids(df, tier):
    result = []
    
    flag = df['tier'] == tier
    summonerId_list = ['summonerId', 'summonerId_1', 'summonerId_2', 'summonerId_3', 'summonerId_4']
    
    target = df[flag]
    
    if tier in ['CHALLENGER', 'GRANDMASTER']:
        pass
    else:
        random_sample = np.random.choice(np.arange(len(target)), size = 1000, replace=False)

        target = target.iloc[random_sample]
    
    api_idx = 0
    cnt = 0
    for idx, row in target.iterrows():
        
        if cnt > 90:
            if api_idx==4:
                api_idx=0
            else:
                api_idx+=1
            
            cnt = 0
                
        summonerId = row[summonerId_list[api_idx]]
        
        try:
            response = tft_summoner_info_summonerId(summonerId, api_idx)
            
            result.append(response.json()['puuid'])
        except:
            print(row)
            print(api_idx)
            print(response)
            break
        cnt+=1
        time.sleep(0.05)

    return result

In [ ]:
tier_puuids = dict()

In [ ]:
tiers = ['CHALLENGER', 'GRANDMASTER', 'MASTER', 'DIAMOND','PLATINUM','GOLD', 'SILVER', 'BRONZE', 'IRON']

In [ ]:
for tier in tqdm(tiers):
    print(tier)
    tier_puuids[tier] = getPuuids(summoners_df, tier)
    
    if tier == 'IRON':
        pass
    else:
        #티어 변경 전 API KEY사용 될 경우 에러 발생으로 대기 시간
        time.sleep(120)

In [ ]:
for key,value in tier_puuids.items():
    puuids_df = pd.DataFrame({
        'puuids' : value
    })
    puuids_df.to_csv('./data/' + key + "_puuids.csv")

In [7]:
puuids_df = pd.read_csv('./data/GOLD_puuids.csv', index_col=0)
puuids_df.head()

,puuids
0,yEILdk7_mAl25DZFe_ttpcx1i38HmO_HU0HH4flg1XaSby...
1,Tq2kf2Zbk0wMSiM9FrcFCkXXcnam8GSAqFeiDHs5ovLf2T...
2,a3BDmYiPVNWf236DiY1jtx5Z8IIDwI_Uh5QQ58vUgX3aVy...
3,pljSbUcPAJYEPD4zkfycdYcdAsoDVJbDC5O6IBLaHT3yYV...
4,bBT_AaaGDbQ7eqtJxMiH_kyGSCAXvskjZ0z8K75jz9Sq0f...


## match_id 가져오기

In [ ]:
tier_match_ids = dict()

In [ ]:
for tier in tiers[6:]:
    df = pd.read_csv('./data/'+tier+"_puuids.csv", index_col=0)
    tier_puuids[tier] = df['puuids'].values

In [ ]:
def getGameIds(puuids, tier):
    result = []
    
    data_su = 50
    
    if tier == 'GRANDMASTER':
        data_su = 20
    elif tier != 'CHALLENGER':
        data_su = 15
    
    api_idx = 0
    api_idxs = [0,1,2,3,4,5,6,7]
    cnt = 0
    for puuid in puuids:
            
        flag = True
        while flag:
            try:
                response = tft_match_ids(puuid, api_idx, data_su)
                if response.status_code != 200:
                    raise
                else:
                    result = result + response.json()
                    flag = False
            except:
                api_idx = np.random.choice(api_idxs, size=1, replace=False)[0]
                time.sleep(1)

        cnt+=1
        time.sleep(0.05)
        
        result = list(set(result))

    return result

In [ ]:
for tier in tqdm(tiers[6:]):
    print(tier)
    tier_match_ids[tier] = getGameIds(tier_puuids[tier], tier)

## 매치 정보 가져오기
- 소환사 게임 기록을 가져옵니다. 매치에는 8명의 소환사가 참가합니다.

In [ ]:
def getGamesDataDf(game_ids):
    result = pd.DataFrame()

    # unix timestamp 
    season8_start_date = '2022-12-07 08:23:00'
    season8_start_date = datetime.datetime.strptime(season8_start_date,'%Y-%m-%d %H:%M:%S').timestamp()

    api_idxs = [0,1,2,3,4,5,6,7]
    api_idx = 0

    for gameId in tqdm(game_ids):
        
        #게임 데이터 가져오기 - 정보 가져올 수 있는 api_key 값 찾기(랜덤 샘플링)
        flag = True
        while flag:
            try:
                    res = tft_get_Game_info(gameId, api_idx)
                    game_data = res.json()['info']
                flag = False
            except:
                api_idx = np.random.choice(api_idxs, size=1, replace=False)[0]
                time.sleep(1)

        # Millisecond -> second
        game_time = (game_data['game_datetime'] / 1000)

        # 시즌 8 게임이 아니라면 패스
        if season8_start_date > game_time:
            continue

        else:
            game_participants = game_data['participants']

            #게임 데이터 추가
            result = pd.concat([result, pd.DataFrame(game_participants)])

#         time.sleep(0.1)

    return result

In [ ]:
for key,value in tqdm(tier_match_ids.items()):
    print(key)
    
    if key in ['CHALLENGER', 'GRANDMASTER','MASTER', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER']:
        continue
    
    game_data =  getGamesDataDf(value)
    game_data.to_csv('./data/' + key + "_games_data.csv")


In [10]:
game_data = pd.read_csv('./data/MASTER_games_data.csv', index_col = 0)
game_data.head()

,augments,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units,partner_group_id
0,"['TFT8_Augment_KayleSupport', 'TFT8_Augment_Du...",{'content_ID': 'd6593334-1e30-4eb1-907c-f64271...,24,26,8,7,0,sss3bYRErAB9ugvi3epX4gR8dXQ7iMmT1JXs8Bq70hakHx...,1549.174316,63,"[{'name': 'Set8_Aegis', 'num_units': 2, 'style...","[{'character_id': 'TFT8_Gangplank', 'itemNames...",NaN
1,"['TFT8_Augment_SylasSupport', 'TFT7_Augment_La...",{'content_ID': '9e1423c2-d982-4db7-b478-f4d723...,7,37,9,1,5,2RwQgzTaRpX07VHH4jIuzLk2075oleDn29NV3VkUNjRGee...,2250.464844,160,"[{'name': 'Set8_Ace', 'num_units': 1, 'style':...","[{'character_id': 'TFT8_Sylas', 'itemNames': [...",NaN
2,"['TFT8_Augment_GangplankSupport', 'TFT6_Augmen...",{'content_ID': 'b890d4df-181a-43da-861d-99a72a...,2,28,8,5,0,amid93D3ipSjwMN8KM1qhsHQHZO_024uzfXttk5eYggzMr...,1699.781738,93,"[{'name': 'Set8_AnimaSquad', 'num_units': 1, '...","[{'character_id': 'TFT8_Gangplank', 'itemNames...",NaN
3,"['TFT8_Augment_WukongSupport', 'TFT6_Augment_B...",{'content_ID': 'fde07006-a93c-45be-a426-34a3d1...,1,37,9,2,1,1Q0b5ipkABE01BUoEdKMKzK3VREbQQIXesiyxwDAchBhLz...,2250.464844,118,"[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Annie', 'itemNames': [...",NaN
4,"['TFT8_Augment_PoppyCarry', 'TFT6_Augment_Item...",{'content_ID': 'b1f08eb3-c601-4ca1-a6c7-15e892...,53,24,8,8,0,n9QWCPL-BqSfXPUPkzwQopW9xbov-_vA_9ztqyMIHIMbP7...,1430.348633,57,"[{'name': 'Set8_Admin', 'num_units': 1, 'style...","[{'character_id': 'TFT8_Lux', 'itemNames': ['T...",NaN
